<a href="https://colab.research.google.com/github/HelloJacob11/TwitterKeyWordExtraction/blob/main/TwitterKeywordExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Twitter KeyWord Extraction
## This program traverses through 1,000 tweets per minute and finds any tweet that has the word "Democrat" or "Republican". Once the program finds tweet with one of those words, it traverses through other tweets that the user of that specific tweet posted to extract any keyword associated with either "Democrat" or "Republican".
- *Project Work Duration: May 2022 - September 2022*

In [ ]:
import tweepy
API_key="Personal API Key"
API_key_secret="API Secret Key"
Bearer_Token"Personal Bearer Token"
Access_Token="Personal Access Token"
Access_Token_Secret="Personal Secret Access Token"
def get_tweets(keyword):
    auth=tweepy.OAuthHandler(API_key,API_key_secret)

    auth.set_access_token(Access_Token,Access_Token_Secret)

    api=tweepy.API(auth)
    #print(api)
    print(keyword)
    ans =[]
    tweets = api.search_tweets(q = keyword,count = 1000)
    for tweet in tweets:
        ans.append(tweet.text)
    return ans

# This function removes all the unnecessary words in a tweet and converts the words in lowercase.
- parameter: text from a tweet
- return: returns a list that contains all the necessary words (words that could be used to extract keywords) in a tweet

In [ ]:
def remove_stopword(text):
   # print(text)
    text=text.replace("\"","")
    text=text.replace("\n"," ")
    text=text.replace("."," ")
    text=text.replace(","," ")
    text=text.replace("!"," ")
    text=text.replace("?"," ")
    split_text=text.split(" ")
    a=open("stopword.txt",'r')
    STOPWORD=a.read().replace("\"","").split(", ")
    All=[]
    for Any in split_text:
        value =  Any.lower()
        che = True
        if len(Any)<=1:
            che=False
        if value not in STOPWORD and che==True:
            for i in value:
                if (i.isalpha()==False) and (i.isdigit()==False):
                    che = False
                    break
            if che == True:
                All.append(value)
    return All

# This function finds how often a specific word appears in the word list.
- parameter: a word and a word list
- return: return a list that contains how often a specific word appears in the word list(parameter).

In [ ]:
def frequency(words,wordlist):
    for word in words:
        if word not in wordlist:
            wordlist[word]=0
        wordlist[word]+=1
    return wordlist



# This function finds the stem of the words in the given word list(parameter). For instance, the stem of fancy is fan.
- parameter: a word list
- return: a list that contains all the "stemmed" words.

In [ ]:
from nltk.stem import PorterStemmer
def stemming(wordList):
    A=stem.PorterStemmer()
    List=[]
    for word in wordList:
        List.append(A.stem(word))
    return List

# This function converts a pos into readable pos.
- parameter: part of speech of a specific word
- return: readable pos.

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
def pos_translate(pos):
    if pos=='J':
        return 'a'
    if pos=="R":
        return 'r'
    if pos=='V':
        return 'v'
    if pos=='N':
        return 'n'



# This function calls pos_translate to convert a pos of a specific word into a readable pos. Then, it "lemmitizes" the word and the readable pos in the word list.  
- parameter: a word list
- return: returns a list that contains all the words that are "lemmitized"

In [ ]:
def lemmatizer_ans(wordList):
    List=[]
    pos_words=pos_tag(wordList)
   # print(pos_words)
    lemmatizer=WordNetLemmatizer()
    for word in pos_words:
        POS=pos_translate(word[1][0])
        if (POS != None):
            List.append(lemmatizer.lemmatize(word[0],POS))
    return List

In [ ]:
import random
def randomWord():
    randomList = [[] for i in range(10)]  #[[],[],[]]
    finalList=[]
    a=open("randomWords.txt",'r')
    randomWords=a.read().split("\n")
    idx = -1
    for a in range(0,len(randomWords),1):
        if len(randomWords[a]) >0 and randomWords[a][0]=="(":
            idx+=1
        else:
            randomList[idx].append(randomWords[a])
    #print(randomList)
    for b in range(0,len(randomList),1):
        value=random.choice(randomList[b])
        finalList.append(value)
    return finalList



# This function processes a list of word from tweets that contain either "republican" or "democrats" to get a list that contains words that can possibly be keywords.
- parameters: a list of word from tweets that contain either "republican" or "democrats", a list that is going to be processed in the function to prevent same word from getting appended, a list that is used for traversing, a list containing words after traversal, a final list of possible candidates for keywords
- return1: return a list of words that are possible candidates of keywords.

In [ ]:
def extraction1(getlistName,total_tweets,tweet_list,ans,out):
    for a in getlistName:
        if a not in total_tweets:
            total_tweets.append(a)
            tweet_list.append(a)
    for tweet in tweet_list:
        word_list = remove_stopword(tweet)
        word_list = lemmatizer_ans(word_list)
        ans = frequency(word_list,ans)

    for word in ans:
        if ans[word]>1:
            out[word] = ans[word]

    dict_word=list(out.items())
    dict_word.sort(key=lambda x:x[1], reverse=True)
    out=dict(dict_word)

    return out

In [ ]:
#parameters: a list that went through extraction1 method, a list that contains random words from various fields of jobs.
#function: this function traverses words in extractedList and removes words that match with random words from randomwordList.
#return: returns a list that contains all the words that are considered keywords

# this function traverses words in extractedList and removes words that match with random words from randomwordList.
- parameters: a list that went through extraction1 method, a list that contains random words from various fields of jobs.
- return: returns a list that contains all the words that are considered keywords

In [ ]:
def extraction2(extractedList,randomwordList):
    mustremovedwordList=[]
    for i in range(len(extractedList),-1,-1):
        for j in range(len(randomwordList),-1,-1):
            if extractedList[i]==randomwordList[j]:
                extractedList.remove(extractedList[i])
    return extractedList

In [ ]:
import time
total_tweetsR = []
total_tweetsD=[]
total_tweetsRA=[]
ansR = {}
ansD={}
ansRA={}
tweet_listR=[]
tweet_listD=[]
tweet_listRA=[]
get_listR=get_tweets("republican")
get_listD=get_tweets("democrat")
Random=randomWord()
get_listRA = []
for f in range(0,10,1):
    get_listRA += get_tweets(Random[f])

outR={}
outD={}
outRA={}
finalR={}
finalD={}
finalRA={}


'''///republicans
for a in get_listR:
    if a not in total_tweetsR:
        total_tweetsR.append(a)
        tweet_listR.append(a)

for tweet in tweet_listR:
    word_listR = remove_stopword(tweet)
    word_listR = lemmatizer_ans(word_listR)
    ans = frequency(word_listR,ansR)

for word in ansR:
    if ansR[word]>1:
        outR[word]=ansR[word]

dictR = list(outR.items())
dictR.sort(key = lambda x: x[1],reverse = True)
outR = dict(dictR)


print("Republican"+" ")
print(outR)
print()
'''
R=extraction1(get_listR,total_tweetsR,tweet_listR,ansR,outR)
R2=extraction2(R,get_listRA)
print(R2)
'''///democrats
for a in get_listD:
    if a not in total_tweetsD:
        total_tweetsD.append(a)
        tweet_listD.append(a)
for tweet in tweet_listD:
   # print(tweet , type(tweet))
    word_listD = remove_stopword(tweet)
    word_listD = lemmatizer_ans(word_listD)
    ansD = frequency(word_listD,ansD)

for word in ansD:
    if ansD[word]>1:
        outD[word] = ansD[word]

dictD=list(outD.items())
dictD.sort(key=lambda x:x[1], reverse=True)
outD=dict(dictD)

print("Democrat"+" ")
print(outD)
print()
'''
'''///random
for a in randomWord():
    if a not in total_tweetsRA:
        total_tweetsRA.append(a)
        tweet_listRA.append(a)

for tweet in tweet_listRA:
    word_listRA=remove_stopword(tweet)
    word_listRA=lemmatizer_ans(word_listRA)
    ansRA=frequency(word_listRA,ansRA)

for word in ansRA:
    if ansRA[word]>1:
        outRA[word] = ansRA[word]

dictRA=list(outRA.items())
dictRA.sort(key=lambda x:x[1], reverse=True)
outRa=dict(dictRA)

print("Random"+" ")
print(outRA)
print()
'''
D=extraction1(get_listD,total_tweetsD,tweet_listD,ansD,outD)
D2=extraction2(D,get_listRA)
print(D2)

republican
democrat
Beach 
Truncated
Collagraphs
Robust
relief
Foundation
flask
crop
Acquittal
Quota


KeyError: 74